[아파트 좌표 데이터](../data/unique_apt_group_xy.csv)

[테스트용](../data/test_origin.csv) 
[훈련용](../data/train_origin.csv)

테스트용, 훈련용 병합해서 결측치 보완
## output
v1.csv

In [1]:
# sys.path를 수정하여 모듈 경로를 설정
import sys
import os
import pandas as pd

# 프로젝트 루트 경로 추가
project_root = os.path.abspath("..")
sys.path.append(project_root)

import module

In [2]:
dt = module.file_load.load_origin_data()

In [3]:
xy_dt = module.file_load.load_apt_xy_file_path()

In [4]:
dt.shape

(1128094, 53)

In [5]:
module.parser.시_군_구_add(dt)

In [6]:
module.parser.아파트_unique_key_add(dt)

In [7]:
dt['좌표X'].isnull().sum(), dt['좌표Y'].isnull().sum()

(876232, 876232)

In [8]:
coords = xy_dt[['아파트_unique_key','좌표X', '좌표Y']]
coords.head(1)

,아파트_unique_key,좌표X,좌표Y
0,강서구_방화동_546-1_방화3차우림필유_2005,126.798318,37.573351


In [9]:
dt = dt.merge(coords, left_on='아파트_unique_key', right_on="아파트_unique_key", how='left')

In [10]:
dt.head(1)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,좌표Y_x,단지신청일,target,is_test,시,구,동,아파트_unique_key,좌표X_y,좌표Y_y
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,37.476763,2022-11-17 10:19:06.0,124000.0,0,서울특별시,강남구,개포동,강남구_개포동_658-1_개포6차우성_1987,127.05721,37.476763


In [11]:
print(dt['좌표X_x'].isnull().sum(), dt['좌표Y_x'].isnull().sum())
print(dt['좌표Y_y'].isnull().sum(), dt['좌표X_y'].isnull().sum())

876232 876232
0 0


In [12]:
# 좌표X_x가 null인 경우 좌표X_y 값으로 채우기
dt['좌표X_x'].fillna(dt['좌표X_y'], inplace=True)

# 좌표Y_x가 null인 경우 좌표Y_y 값으로 채우기
dt['좌표Y_x'].fillna(dt['좌표Y_y'], inplace=True)

In [13]:
print(dt['좌표X_x'].isnull().sum(), dt['좌표Y_x'].isnull().sum())
print(dt['좌표Y_y'].isnull().sum(), dt['좌표X_y'].isnull().sum())

0 0
0 0


In [14]:
# '_x'와 '_y' 열 정리
dt.rename(columns={'좌표X_x': '좌표X', '좌표Y_x': '좌표Y'}, inplace=True)
dt.drop(['좌표X_y', '좌표Y_y', '시','구','동', '아파트_unique_key'], axis=1, inplace=True)

In [15]:
dt['좌표X'].isnull().sum()
dt['좌표Y'].isnull().sum()

0

In [16]:
dt.query('is_test == 0').shape, dt.query('is_test == 1').shape

((1118822, 53), (9272, 53))

In [17]:
module.save.save_version(dt, 'v1')

/data/ephemeral/home/local_git/dev_collect/module/save.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_train.drop(['is_test'], axis=1, inplace=True)
/data/ephemeral/home/local_git/dev_collect/module/save.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_test.drop(['is_test'], axis=1, inplace=True)


v1.csv 파일이 저장되었습니다.
